<a href="https://colab.research.google.com/github/GerardoFesta/3DFER_SE4AI/blob/1-poor-performances-on-test-set/ResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn

In [2]:
%matplotlib inline
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from datetime import datetime
import pandas as pd
import cv2
from google.colab.patches import cv2_imshow

TRAIN_PATH = "train"
TEST_PATH ="test"


In [ ]:
import os 
from google.colab import drive
drive.mount("/content/drive")

MessageError: ignored

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#Questo commento serve per provare il funzionamento dei commit con colab

batch_size = 1024 

In [6]:
import zipfile


zip_ref = zipfile.ZipFile('fer2013.zip', 'r') #Opens the zip file in read mode
zip_ref.extractall() #Extracts the files into the /tmp folder
zip_ref.close()

In [7]:
import os
class_dir=os.listdir(TRAIN_PATH+'/')
folder_names = class_dir
label_dict = {folder_names[i]:i for i in range(len(folder_names))}
num_classes=len(label_dict)
train_image_filenames = []
train_labels = []

# iterate through each folder and collect filenames and labels
for folder_name in folder_names:
    folder_path = os.path.join(TRAIN_PATH, folder_name)
    for filename in os.listdir(folder_path):
        train_image_filenames.append(os.path.join(TRAIN_PATH+"/"+folder_name, filename))
        train_labels.append(label_dict[folder_name])

# create pandas dataframe
train_df = pd.DataFrame({'filename': train_image_filenames, 'emotion': train_labels})

print(len(train_df))


test_image_filenames = []
test_labels = []
class_dir=os.listdir(TEST_PATH+'/')
for folder_name in folder_names:
    folder_path = os.path.join(TEST_PATH, folder_name)
    for filename in os.listdir(folder_path):
        test_image_filenames.append(os.path.join(TEST_PATH+"/"+folder_name, filename))
        test_labels.append(label_dict[folder_name])

test_df = pd.DataFrame({'filename': test_image_filenames, 'emotion': test_labels})

28709


In [8]:
train_df['img_as_matrix'] = train_df['filename'].apply(lambda path: cv2.imread(path))
test_df['img_as_matrix'] = test_df['filename'].apply(lambda path: cv2.imread(path))


In [9]:
class CustomDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.data = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        image = self.data.iloc[index]['img_as_matrix'] 
        label = self.data.iloc[index]['emotion'] 

        # Esegui le trasformazioni se definite
        if self.transform is not None:
            image = self.transform(image)

        return image, label

In [10]:
train_df.drop(columns=["filename"], inplace=True)
test_df.drop(columns=["filename"], inplace=True)
train_df.at[0,"img_as_matrix"]
train_dataset = CustomDataset(train_df, transform=transforms.ToTensor())
test_dataset = CustomDataset(test_df, transform=transforms.ToTensor())
 

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [15]:
def accuracy(preds, labels):
  probabilities = torch.nn.functional.softmax(preds, dim=1)
  _, predicted = torch.max(probabilities, dim=1)
  print("PREDICTED ", predicted)
  print("LABELS ", labels) 
  n_correct = (predicted==labels).sum().float()

  acc =n_correct / labels.shape[0]
  acc= torch.round(acc*100)
  return acc;

# Resnet 50 Pytorch Code

In [16]:
class block(nn.Module):
    def __init__(
        self, in_channels, intermediate_channels, identity_downsample=None, stride=1
    ):
        super().__init__()
        self.expansion = 4
        self.conv1 = nn.Conv2d(
            in_channels,
            intermediate_channels,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=False,
        )
        self.bn1 = nn.BatchNorm2d(intermediate_channels)
        self.conv2 = nn.Conv2d(
            intermediate_channels,
            intermediate_channels,
            kernel_size=3,
            stride=stride,
            padding=1,
            bias=False,
        )
        self.bn2 = nn.BatchNorm2d(intermediate_channels)
        self.conv3 = nn.Conv2d(
            intermediate_channels,
            intermediate_channels * self.expansion,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=False,
        )
        self.bn3 = nn.BatchNorm2d(intermediate_channels * self.expansion)
        self.relu = nn.ReLU()
        self.identity_downsample = identity_downsample
        self.stride = stride

    def forward(self, x):
        identity = x.clone()

        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.conv3(x)
        x = self.bn3(x)

        if self.identity_downsample is not None:
            identity = self.identity_downsample(identity)

        x += identity
        x = self.relu(x)
        return x


class ResNet(nn.Module):
    def __init__(self, block, layers, image_channels, num_classes):
        super(ResNet, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(
            image_channels, 64, kernel_size=7, stride=2, padding=3, bias=False
        )
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # Essentially the entire ResNet architecture are in these 4 lines below
        self.layer1 = self._make_layer(
            block, layers[0], intermediate_channels=64, stride=1
        )
        self.layer2 = self._make_layer(
            block, layers[1], intermediate_channels=128, stride=2
        )
        self.layer3 = self._make_layer(
            block, layers[2], intermediate_channels=256, stride=2
        )
        self.layer4 = self._make_layer(
            block, layers[3], intermediate_channels=512, stride=2
        )

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc1 = nn.Linear(512 * 4, 512)
        self.bn2 = nn.BatchNorm1d(512)
        self.fc2 = nn.Linear(512, 256)
        self.bn3 = nn.BatchNorm1d(256)        
        self.fc3 = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc1(x)
        x = self.bn2(x)
        x = self.fc2(x)
        x = self.bn3(x)
        x = self.fc3(x)

        return x

    def _make_layer(self, block, num_residual_blocks, intermediate_channels, stride):
        identity_downsample = None
        layers = []

        # Either if we half the input space for ex, 56x56 -> 28x28 (stride=2), or channels changes
        # we need to adapt the Identity (skip connection) so it will be able to be added
        # to the layer that's ahead
        if stride != 1 or self.in_channels != intermediate_channels * 4:
            identity_downsample = nn.Sequential(
                nn.Conv2d(
                    self.in_channels,
                    intermediate_channels * 4,
                    kernel_size=1,
                    stride=stride,
                    bias=False,
                ),
                nn.BatchNorm2d(intermediate_channels * 4),
            )

        layers.append(
            block(self.in_channels, intermediate_channels, identity_downsample, stride)
        )

        # The expansion size is always 4 for ResNet 50,101,152
        self.in_channels = intermediate_channels * 4

        # For example for first resnet layer: 256 will be mapped to 64 as intermediate layer,
        # then finally back to 256. Hence no identity downsample is needed, since stride = 1,
        # and also same amount of channels.
        for i in range(num_residual_blocks - 1):
            layers.append(block(self.in_channels, intermediate_channels))

        return nn.Sequential(*layers)


def ResNet50(img_channel=3, num_classes=7):
    return ResNet(block, [3, 4, 6, 3], img_channel, num_classes)



In [17]:
model = ResNet50().to(device)

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [18]:
n_total_steps = len(train_loader)
num_epochs = 20
for epoch in range(num_epochs):

    running_loss = 0.0
    running_acc = 0.0

    for i, (images, labels) in enumerate(train_loader):
       
      images = images.to(device)
      labels = labels.to(device)

      outputs = model(images)

      loss = criterion(outputs, labels)
      

      acc = accuracy(outputs, labels)

      optimizer.zero_grad()      
      loss.backward()            
      optimizer.step()  
      running_loss += loss.item()    
      running_acc += acc.item()

    print (f'Epoch [{epoch}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Acc: {running_acc/len(train_loader):.4f}')

print('Finished Training')

PATH = './cnn.pth'
torch.save(model.state_dict(), PATH)

PREDICTED  tensor([2, 0, 4,  ..., 4, 4, 3], device='cuda:0')
LABELS  tensor([2, 5, 4,  ..., 0, 3, 4], device='cuda:0')
PREDICTED  tensor([4, 4, 6,  ..., 0, 4, 1], device='cuda:0')
LABELS  tensor([5, 0, 4,  ..., 2, 0, 6], device='cuda:0')
PREDICTED  tensor([6, 5, 1,  ..., 5, 5, 4], device='cuda:0')
LABELS  tensor([2, 2, 2,  ..., 0, 0, 4], device='cuda:0')
PREDICTED  tensor([5, 5, 4,  ..., 5, 4, 6], device='cuda:0')
LABELS  tensor([0, 5, 3,  ..., 4, 4, 5], device='cuda:0')
PREDICTED  tensor([1, 5, 2,  ..., 1, 1, 6], device='cuda:0')
LABELS  tensor([5, 1, 4,  ..., 5, 2, 4], device='cuda:0')
PREDICTED  tensor([5, 4, 4,  ..., 4, 3, 3], device='cuda:0')
LABELS  tensor([2, 5, 2,  ..., 4, 3, 4], device='cuda:0')
PREDICTED  tensor([6, 2, 2,  ..., 0, 0, 1], device='cuda:0')
LABELS  tensor([2, 6, 2,  ..., 2, 5, 4], device='cuda:0')
PREDICTED  tensor([4, 4, 0,  ..., 3, 0, 3], device='cuda:0')
LABELS  tensor([2, 4, 0,  ..., 0, 5, 4], device='cuda:0')
PREDICTED  tensor([4, 0, 4,  ..., 3, 6, 4], devi

In [19]:
with torch.no_grad():
    
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
       
        outputs = model(images)
        acc=accuracy(outputs, labels)
        print("ACC ON TEST SET: ", acc)

PREDICTED  tensor([0, 0, 2,  ..., 2, 2, 5], device='cuda:0')
LABELS  tensor([0, 0, 0,  ..., 0, 0, 0], device='cuda:0')
ACC ON TEST SET:  tensor(18., device='cuda:0')
PREDICTED  tensor([2, 2, 3,  ..., 4, 4, 5], device='cuda:0')
LABELS  tensor([0, 0, 0,  ..., 2, 2, 2], device='cuda:0')
ACC ON TEST SET:  tensor(33., device='cuda:0')
PREDICTED  tensor([4, 5, 4,  ..., 2, 0, 4], device='cuda:0')
LABELS  tensor([2, 2, 2,  ..., 2, 2, 2], device='cuda:0')
ACC ON TEST SET:  tensor(33., device='cuda:0')
PREDICTED  tensor([6, 2, 2,  ..., 2, 4, 0], device='cuda:0')
LABELS  tensor([2, 2, 2,  ..., 4, 4, 4], device='cuda:0')
ACC ON TEST SET:  tensor(22., device='cuda:0')
PREDICTED  tensor([2, 4, 2,  ..., 4, 4, 2], device='cuda:0')
LABELS  tensor([4, 4, 4,  ..., 4, 4, 4], device='cuda:0')
ACC ON TEST SET:  tensor(20., device='cuda:0')
PREDICTED  tensor([2, 0, 6,  ..., 5, 4, 2], device='cuda:0')
LABELS  tensor([4, 4, 4,  ..., 5, 5, 5], device='cuda:0')
ACC ON TEST SET:  tensor(15., device='cuda:0')
PRED